# **Bierproduktion 2**

Ein Unternehmen stellt fünf unterschiedliche Biersorten her und möchte durch Optimierung der Produktionsmengen den möglichen Deckungsbeitrag maximieren. Der Deckungsbeitrag der durch die unterschiedlichen Biersorten erzielt werden kann ist in folgender Tabelle aufgeführt:

| Sorte  | DB je Liter|
|----------| ------|
| A       |  2,10 € |
| B       |  1,70 € |
| C       |  2,00 € |
| D       |  1,50 € |
| E       |  1,85 € |


In der nachstehenden Tabelle sind sowohl die Abfülldauer (AD) als auch die Rohstoffverbräuche (RV) für einen Liter der jeweiligen Biersorten ausgewiesen:

| Sorte  | AD | RV1 | RV2 | RV3 |
|---------|------|-------|-----|-----|
| A       |  2,0 | 1,6  | 1,6 | 1,0   |
| B       |  1,3 | 1,7  | 1,0  | 1,0   |
| C       |  1,3 | 1,4 | 1,35  | 1,4 |
| D       |  1,0 | 1,3  | 2,1  | 1,6 |
| E       |  1,9 | 1,0  | 1,0  | 1,4|


Die Abfüllung der unterschiedlichen Sorten findet auf denselben Anlagen statt. Theoretisch ist unter Wahrung der zeitlichen Kapazität eine tägliche Abfüllmenge von 4000 l der Biersorte A möglich, wenn keine weitere Sorte produziert wird. Die tägliche Belieferung der Rohstoffe 1, 2 und 3 sieht wie folgt aus:

| Rohstoff  | Einheiten |
|--------|-------|
| 1      |  7500 | 
| 2      |  7000 |
| 3      |  6000 | 


Die täglich maximal nachgefragte Menge nach den Produkten ist kostant und unterschiedet sich ausschließlich zwischen den verschieden Sorten. Die maximale Nachfrage nach den jeweiligen Sorten zeigt die nachstehende Tabelle:

| Sorte  | max. Nachfrage|
|----------| ------|
| A       |  2100 l  |
| B       |  1100 l  |
| C       |  1100 l  |
| D       |  1600 l  |
| E       |  1600 l  |


Da die Biersorten B und C sowie die Sorten A und D jeweils dieselbe Zielgruppe ansprechen, sollen insgesamt maximal 1400 l von B und C bzw. 2700 l von A und D produziert werden, um das unternehmerische Risiko durch Absatzrückgänge zu senken. 

**Aufgabe: Beschreiben Sie das Produktionproblem durch ein lineares Optimierungsmodell und ermitteln Sie die optimalen täglichen Produktionsmengen der einzelnen Biersorten.**

In [9]:
import pulp

**1. Initialisierung des Optimierungsmodells und der Entscheidungsvariablen**

Die Entscheidungsvariablen repräsentieren die täglichen Produktionsmengen (in Litern) der fünf Biersorten:

- $x_1$ = Produktionsmenge Biersorte A (in Liter)
- $x_2$ = Produktionsmenge Biersorte B (in Liter)
- $x_3$ = Produktionsmenge Biersorte C (in Liter)
- $x_4$ = Produktionsmenge Biersorte D (in Liter)
- $x_5$ = Produktionsmenge Biersorte E (in Liter)

Alle Variablen sind kontinuierlich und nicht-negativ ($x_i \geq 0$).

In [10]:
lp_problem = pulp.LpProblem("LPProblem", pulp.LpMaximize)

x1 = pulp.LpVariable('x1', lowBound=0, cat='Continuous')  # Biersorte A
x2 = pulp.LpVariable('x2', lowBound=0, cat='Continuous')  # Biersorte B
x3 = pulp.LpVariable('x3', lowBound=0, cat='Continuous')  # Biersorte C
x4 = pulp.LpVariable('x4', lowBound=0, cat='Continuous')  # Biersorte D
x5 = pulp.LpVariable('x5', lowBound=0, cat='Continuous')  # Biersorte E

**2. Definition der Zielfunktion**

Die Zielfunktion maximiert den gesamten Deckungsbeitrag (DB) aus allen fünf Biersorten:

$$Z = 2{,}10 \cdot x_1 + 1{,}70 \cdot x_2 + 2{,}00 \cdot x_3 + 1{,}50 \cdot x_4 + 1{,}85 \cdot x_5 \rightarrow \max$$

In [11]:
# Zielfunktion: Deckungsbeitrag maximieren
lp_problem += 2.10 * x1 + 1.70 * x2 + 2.00 * x3 + 1.50 * x4 + 1.85 * x5, "Z"

**3. Definition der problemspezifischen Restriktionen**

**Kapazitäts- und Rohstoffrestriktionen:**

Die Abfüllung aller Sorten findet auf denselben Anlagen statt. Bei ausschließlicher Produktion von Sorte A (AD = 2,0) könnten 4.000 l abgefüllt werden → Gesamtkapazität = $4.000 \times 2{,}0 = 8.000$ Zeiteinheiten.

$$2{,}0 \cdot x_1 + 1{,}3 \cdot x_2 + 1{,}3 \cdot x_3 + 1{,}0 \cdot x_4 + 1{,}9 \cdot x_5 \leq 8.000$$

Die tägliche Belieferung der drei Rohstoffe begrenzt die Produktion zusätzlich:

- RV1: $1{,}6 \cdot x_1 + 1{,}7 \cdot x_2 + 1{,}4 \cdot x_3 + 1{,}3 \cdot x_4 + 1{,}0 \cdot x_5 \leq 7.500$
- RV2: $1{,}6 \cdot x_1 + 1{,}0 \cdot x_2 + 1{,}35 \cdot x_3 + 2{,}1 \cdot x_4 + 1{,}0 \cdot x_5 \leq 7.000$
- RV3: $1{,}0 \cdot x_1 + 1{,}0 \cdot x_2 + 1{,}4 \cdot x_3 + 1{,}6 \cdot x_4 + 1{,}4 \cdot x_5 \leq 6.000$

In [12]:
# Restriktion Abfülldauer (Kapazität: 4000 l * AD_A = 4000 * 2.0 = 8000 ZE)
lp_problem += 2.0 * x1 + 1.3 * x2 + 1.3 * x3 + 1.0 * x4 + 1.9 * x5 <= 8000
# Restriktion Rohstoff 1
lp_problem += 1.6 * x1 + 1.7 * x2 + 1.4 * x3 + 1.3 * x4 + 1.0 * x5 <= 7500
# Restriktion Rohstoff 2
lp_problem += 1.6 * x1 + 1.0 * x2 + 1.35 * x3 + 2.1 * x4 + 1.0 * x5 <= 7000
# Restriktion Rohstoff 3
lp_problem += 1.0 * x1 + 1.0 * x2 + 1.4 * x3 + 1.6 * x4 + 1.4 * x5 <= 6000

**Nachfragerestriktionen:**

Die täglich maximal nachgefragte Menge ist für jede Sorte begrenzt:

- $x_1 \leq 2100$, $x_2 \leq 1100$, $x_3 \leq 1100$, $x_4 \leq 1600$, $x_5 \leq 1600$

In [13]:
# Nachfragerestriktionen (maximale tägliche Nachfrage je Sorte)
lp_problem += x1 <= 2100  # Sorte A
lp_problem += x2 <= 1100  # Sorte B
lp_problem += x3 <= 1100  # Sorte C
lp_problem += x4 <= 1600  # Sorte D
lp_problem += x5 <= 1600  # Sorte E

**Gruppenrestriktionen (Zielgruppenrisiko):**

Die Sorten B und C sowie A und D sprechen jeweils dieselbe Zielgruppe an. Um das Absatzrisiko zu senken:

- $x_2 + x_3 \leq 1400$ (B und C zusammen)
- $x_1 + x_4 \leq 2700$ (A und D zusammen)

In [14]:
# Gruppenrestriktionen (Zielgruppenrisiko)
lp_problem += x2 + x3 <= 1400  # Sorten B und C zusammen max. 1400 l
lp_problem += x1 + x4 <= 2700  # Sorten A und D zusammen max. 2700 l

**4. Generierung der optimalen Lösung**

In [15]:
print(lp_problem)

lp_problem.solve()
print("Status:", pulp.LpStatus[lp_problem.status])

for variable in lp_problem.variables():
    print("{} = {}".format(variable.name, variable.varValue))

print("Deckungsbeitrag =",pulp.value(lp_problem.objective))

LPProblem:
MAXIMIZE
2.1*x1 + 1.7*x2 + 2.0*x3 + 1.5*x4 + 1.85*x5 + 0.0
SUBJECT TO
_C1: 2 x1 + 1.3 x2 + 1.3 x3 + x4 + 1.9 x5 <= 8000

_C2: 1.6 x1 + 1.7 x2 + 1.4 x3 + 1.3 x4 + x5 <= 7500

_C3: 1.6 x1 + x2 + 1.35 x3 + 2.1 x4 + x5 <= 7000

_C4: x1 + x2 + 1.4 x3 + 1.6 x4 + 1.4 x5 <= 6000

_C5: x1 <= 2100

_C6: x2 <= 1100

_C7: x3 <= 1100

_C8: x4 <= 1600

_C9: x5 <= 1600

_C10: x2 + x3 <= 1400

_C11: x1 + x4 <= 2700

VARIABLES
x1 Continuous
x2 Continuous
x3 Continuous
x4 Continuous
x5 Continuous

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/ridwan/Desktop/PROJECTS/LEUPHANA/ASCM-ws2526-Gruppe-1/.venv/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/1fb8fef02929494a8f6d5f686d5bdc54-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/1fb8fef02929494a8f6d5f686d5bdc54-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 16 COLUMNS
At line 51 RHS
At line 63 BOUNDS
At l

**Aufgabe: Wie verändert sich das Optimierungsmodell, wenn aus prozesstechnischen Gründen mindestens 500 l der einzlnen Biersorten produziert werden müssen? Hat die Veränderung Auswirkungen auf die optimale Lösung?**

Aus prozesstechnischen Gründen müssen mindestens 500 l jeder Biersorte produziert werden. Das Modell wird angepasst, indem die Untergrenze der Entscheidungsvariablen von 0 auf 500 erhöht wird:

$$x_i \geq 500 \quad \forall \, i \in \{1, 2, 3, 4, 5\}$$

Alle anderen Restriktionen und die Zielfunktion bleiben unverändert. Durch die zusätzliche Mindestmenge wird der zulässige Lösungsraum eingeschränkt, was zu einem geringeren maximalen Deckungsbeitrag führen kann.

In [17]:
# Neues Modell mit Mindestproduktionsmenge von 500 l je Sorte
lp_problem_2 = pulp.LpProblem("LPProblem_MinProd", pulp.LpMaximize)

# Entscheidungsvariablen mit lowBound=500 (Mindestproduktion)
x1_b = pulp.LpVariable('x1', lowBound=500, cat='Continuous')  # Sorte A
x2_b = pulp.LpVariable('x2', lowBound=500, cat='Continuous')  # Sorte B
x3_b = pulp.LpVariable('x3', lowBound=500, cat='Continuous')  # Sorte C
x4_b = pulp.LpVariable('x4', lowBound=500, cat='Continuous')  # Sorte D
x5_b = pulp.LpVariable('x5', lowBound=500, cat='Continuous')  # Sorte E

# Zielfunktion
lp_problem_2 += 2.10 * x1_b + 1.70 * x2_b + 2.00 * x3_b + 1.50 * x4_b + 1.85 * x5_b, "Z"

# Restriktion Abfülldauer
lp_problem_2 += 2.0 * x1_b + 1.3 * x2_b + 1.3 * x3_b + 1.0 * x4_b + 1.9 * x5_b <= 8000
# Restriktion Rohstoff 1
lp_problem_2 += 1.6 * x1_b + 1.7 * x2_b + 1.4 * x3_b + 1.3 * x4_b + 1.0 * x5_b <= 7500
# Restriktion Rohstoff 2
lp_problem_2 += 1.6 * x1_b + 1.0 * x2_b + 1.35 * x3_b + 2.1 * x4_b + 1.0 * x5_b <= 7000
# Restriktion Rohstoff 3
lp_problem_2 += 1.0 * x1_b + 1.0 * x2_b + 1.4 * x3_b + 1.6 * x4_b + 1.4 * x5_b <= 6000

# Nachfragerestriktionen
lp_problem_2 += x1_b <= 2100
lp_problem_2 += x2_b <= 1100
lp_problem_2 += x3_b <= 1100
lp_problem_2 += x4_b <= 1600
lp_problem_2 += x5_b <= 1600

# Gruppenrestriktionen
lp_problem_2 += x2_b + x3_b <= 1400
lp_problem_2 += x1_b + x4_b <= 2700

# Lösung
print(lp_problem_2)

lp_problem_2.solve()
print("\nStatus:", pulp.LpStatus[lp_problem_2.status])

for variable in lp_problem_2.variables():
    print("{} = {}".format(variable.name, variable.varValue))

print("\nDeckungsbeitrag (mit Mindestmenge) =", pulp.value(lp_problem_2.objective))

LPProblem_MinProd:
MAXIMIZE
2.1*x1 + 1.7*x2 + 2.0*x3 + 1.5*x4 + 1.85*x5 + 0.0
SUBJECT TO
_C1: 2 x1 + 1.3 x2 + 1.3 x3 + x4 + 1.9 x5 <= 8000

_C2: 1.6 x1 + 1.7 x2 + 1.4 x3 + 1.3 x4 + x5 <= 7500

_C3: 1.6 x1 + x2 + 1.35 x3 + 2.1 x4 + x5 <= 7000

_C4: x1 + x2 + 1.4 x3 + 1.6 x4 + 1.4 x5 <= 6000

_C5: x1 <= 2100

_C6: x2 <= 1100

_C7: x3 <= 1100

_C8: x4 <= 1600

_C9: x5 <= 1600

_C10: x2 + x3 <= 1400

_C11: x1 + x4 <= 2700

VARIABLES
500 <= x1 Continuous
500 <= x2 Continuous
500 <= x3 Continuous
500 <= x4 Continuous
500 <= x5 Continuous

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/ridwan/Desktop/PROJECTS/LEUPHANA/ASCM-ws2526-Gruppe-1/.venv/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/5892419633944be0ac10c9b599645d03-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/5892419633944be0ac10c9b599645d03-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 16 CO

## **Zusammenfassung**

### Optimale tägliche Produktionsmengen

| Sorte | Produktionsmenge | Deckungsbeitrag je Liter | DB-Beitrag |
|-------|----------------:|------------------------:|-----------:|
| A ($x_1$) | 1.899,48 l | 2,10 € | 3.988,91 € |
| B ($x_2$) | 300,00 l | 1,70 € | 510,00 € |
| C ($x_3$) | 1.100,00 l | 2,00 € | 2.200,00 € |
| D ($x_4$) | 586,30 l | 1,50 € | 879,45 € |
| E ($x_5$) | 944,60 l | 1,85 € | 1.747,51 € |
| **Gesamt** | **4.830,38 l** | | **9.325,87 €** |

### Mindestproduktion von 500 l je Sorte

| Sorte | Ohne Mindestmenge | Mit Mindestmenge (≥ 500 l) | Veränderung |
|-------|------------------:|---------------------------:|------------:|
| A ($x_1$) | 1.899,48 l | 1.845,72 l | −53,76 l |
| B ($x_2$) | 300,00 l | 500,00 l | **+200,00 l** |
| C ($x_3$) | 1.100,00 l | 900,00 l | −200,00 l |
| D ($x_4$) | 586,30 l | 649,48 l | +63,18 l |
| E ($x_5$) | 944,60 l | 967,93 l | +23,33 l |
| **DB** | **9.325,87 €** | **9.290,92 €** | **−34,95 €** |


**Es ist deutlich, das die Mindestproduktionsmenge hat Auswirkungen auf die optimale Lösung.** Der maximale Deckungsbeitrag sinkt um ca. 34,95 € (von 9.325,87 € auf 9.290,92 €). Die wesentliche Veränderung betrifft Biersorte B: Im Originalmodell werden nur 300 l produziert, was unter der neuen Mindestmenge von 500 l liegt. Dadurch wird die Produktion von B auf 500 l erhöht, während die Sorten A und C ihre Mengen reduzieren, um die Kapazitäts- und Rohstoffrestriktionen weiterhin einzuhalten.